In [ ]:
import torch,math
from torch import tensor,nn,optim
from torch.utils.data import DataLoader,default_collate
from torch.nn import init
from functools import partial
import torchvision.transforms.functional as TF,torch.nn.functional as F
from scipy import linalg, integrate
from datasets import load_dataset
from diffusers.models.attention_processor import Attention
from einops import rearrange

In [ ]:
def_device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

# autoencoder
def conv(ni, nf, ks=3, stride=2, act=True):
    res = nn.Conv2d(ni, nf, stride=stride, kernel_size=ks, padding=ks//2)
    if act: res = nn.Sequential(res, nn.ReLU())
    return res

def deconv(ni, nf, ks=3, act=True):
    layers = [nn.ConvTranspose2d(ni, nf, kernel_size=ks, stride=2, padding=ks//2)]
    if act: layers.append(nn.ReLU())
    return nn.Sequential(*layers)

ae = nn.Sequential(   #28x28
    nn.ZeroPad2d(2),  #32x32
    conv(1,2),        #16x16
    conv(2,4),        #8x8
#     conv(4,8),        #4x4
#     deconv(8,4),      #8x8
    deconv(4,2),      #16x16
    deconv(2,1, act=False), #32x32
    nn.ZeroPad2d(-2), #28x28
    nn.Sigmoid()
).to(def_device)

ni,nh,nl = 784,400,200
def lin(ni, nf, act=nn.SiLU, norm=nn.BatchNorm1d, bias=True):
    layers = nn.Sequential(nn.Linear(ni, nf, bias=bias))
    if act : layers.append(act())
    if norm: layers.append(norm(nf))
    return layers

def init_weights(m, leaky=0.):
    if isinstance(m, (nn.Conv1d,nn.Conv2d,nn.Conv3d,nn.Linear)): init.kaiming_normal_(m.weight, a=leaky)

iw = partial(init_weights, leaky=0.2)

class Autoenc(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc = nn.Sequential(lin(ni, nh), lin(nh, nh), lin(nh, nl))
        self.dec = nn.Sequential(lin(nl, nh), lin(nh, nh), lin(nh, ni, act=None))
        iw(self)
        
    def forward(self, x):
        x = self.enc(x)
        return self.dec(x)

# VAE (Variational ae), z is latent variable
class VAE(nn.Module): 
    def __init__(self, input_dim, hidden_dim, latent_dim): 
        super(VAE, self).__init__() 
        # 编码器部分 
        self.fc1  = nn.Linear(input_dim, hidden_dim) 
        self.fc_mu  = nn.Linear(hidden_dim, latent_dim) 
        self.fc_logvar  = nn.Linear(hidden_dim, latent_dim) 
        # 解码器部分 
        self.fc2  = nn.Linear(latent_dim, hidden_dim) 
        self.fc3  = nn.Linear(hidden_dim, input_dim) 

    def encode(self, x): 
        h = torch.relu(self.fc1(x))  
        mu = self.fc_mu(h)  
        logvar = self.fc_logvar(h)  
        return mu, logvar 
 
    def reparameterize(self, mu, logvar): 
        std = torch.exp(0.5  * logvar) 
        eps = torch.randn_like(std)  
        return mu + eps * std 
 
    def decode(self, z): 
        h = torch.relu(self.fc2(z))  
        return torch.sigmoid(self.fc3(h))  
 
    def forward(self, x): 
        mu, logvar = self.encode(x)  
        z = self.reparameterize(mu,  logvar) 
        return self.decode(z),  mu, logvar 

class VAE2(nn.Module):
    def __init__(self):
        super().__init__()
        self.enc = nn.Sequential(lin(ni, nh), lin(nh, nh))
        self.mu,self.lv = lin(nh, nl, act=None),lin(nh, nl, act=None)
        self.dec = nn.Sequential(lin(nl, nh), lin(nh, nh), lin(nh, ni, act=None))
        iw(self)
        
    def forward(self, x):
        x = self.enc(x)
        mu,lv = self.mu(x),self.lv(x)
        z = mu + (0.5*lv).exp()*torch.randn_like(lv)
        return self.dec(z),mu,lv

# Kullback-Leibler Divergence Loss
def kld_loss(inp, x):
    x_hat,mu,lv = inp
    return -0.5 * (1 + lv - mu.pow(2) - lv.exp()).mean()
# Binary Cross-Entropy Loss
def bce_loss(inp, x): return F.binary_cross_entropy_with_logits(inp[0], x)

def vae_loss(inp, x): return kld_loss(inp, x) + bce_loss(inp,x)


In [ ]:
class GeneralRelu(nn.Module):
    def __init__(self, leak=None, sub=None, maxv=None):
        super().__init__()
        self.leak,self.sub,self.maxv = leak,sub,maxv

    def forward(self, x): 
        x = F.leaky_relu(x,self.leak) if self.leak is not None else F.relu(x)
        if self.sub is not None: x -= self.sub
        if self.maxv is not None: x.clamp_max_(self.maxv)
        return x
def noop (x=None, *args, **kwargs):
    "Do nothing"
    return x

In [ ]:
# RESNET
act_gr = partial(GeneralRelu, leak=0.1, sub=0.4)

def _conv_block(ni, nf, stride, act=act_gr, norm=None, ks=3):
    return nn.Sequential(conv(ni, nf, stride=1, act=act, norm=norm, ks=ks),
                         conv(nf, nf, stride=stride, act=None, norm=norm, ks=ks))

class ResBlock(nn.Module):
    def __init__(self, ni, nf, stride=1, ks=3, act=act_gr, norm=None):
        super().__init__()
        self.convs = _conv_block(ni, nf, stride, act=act, ks=ks, norm=norm)
        self.idconv = noop if ni==nf else conv(ni, nf, ks=1, stride=1, act=None)
        self.pool = noop if stride==1 else nn.AvgPool2d(2, ceil_mode=True)
        self.act = act()

    def forward(self, x): return self.act(self.convs(x) + self.idconv(self.pool(x)))

def get_model(act=nn.ReLU, nfs=(8,16,32,64,128,256), norm=nn.BatchNorm2d):
    layers = [ResBlock(1, 8, stride=1, act=act, norm=norm)]
    layers += [ResBlock(nfs[i], nfs[i+1], act=act, norm=norm, stride=2) for i in range(len(nfs)-1)]
    layers += [nn.Flatten(), nn.Linear(nfs[-1], 10, bias=False), nn.BatchNorm1d(10)]
    return nn.Sequential(*layers).to(def_device)

# ResNet类的基本结构
# ResNet类通常继承自PyTorch的nn.Module基类，包含__init__初始化方法和forward前向传播方法。在__init__中定义网络各层结构，forward方法则描述数据在网络中的流动顺序。
# ResNet主要由初始卷积层、批量归一化层、ReLU激活函数、最大池化层、4个残差块层（layer1-layer4）、平均池化层和全连接层组成。
# 残差块实现
# ResNet包含两种残差块：BasicBlock和Bottleneck。BasicBlock用于较浅的网络（如ResNet18/34），由两个3x3卷积层组成；Bottleneck用于深层网络（如ResNet50/101/152），由1x1、3x3、1x1三个卷积层构成，
# 第一个1x1卷积用于降维，最后一个1x1卷积恢复维度。每个卷积层后都跟随批量归一化和ReLU激活。残差连接通过将输入直接加到输出上来实现，当维度不匹配时使用1x1卷积进行下采样。
# forward方法实现
# forward方法定义了数据流动路径：先通过初始卷积层（conv1）、批量归一化（bn1）、ReLU和最大池化（maxpool），然后依次通过4个残差块层（layer1-layer4），最后经过平均池化（avgpool）、展平（flatten）和全连接层（fc）。
# 在PyTorch中，可以直接通过实例调用（如model(input)）来自动触发forward方法，这得益于Python的__call__机制。
class BasicBlock(nn.Module):
    expansion = 1 
    
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1  = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1  = nn.BatchNorm2d(planes)
        self.conv2  = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2  = nn.BatchNorm2d(planes)
        
        self.shortcut  = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes: 
            self.shortcut  = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,  kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes) 
            )
            
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x))) 
        out = self.bn2(self.conv2(out)) 
        out += self.shortcut(x) 
        out = F.relu(out) 
        return out 
 
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_planes  = 64 
        
        self.conv1  = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1  = nn.BatchNorm2d(64)
        self.maxpool  = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1  = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2  = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3  = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4  = self._make_layer(block, 512, num_blocks[3], stride=2)
        
        self.avgpool  = nn.AdaptiveAvgPool2d((1, 1))
        self.fc  = nn.Linear(512*block.expansion,  num_classes)
        
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes,  planes, stride))
            self.in_planes  = planes * block.expansion  
        return nn.Sequential(*layers)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x))) 
        x = self.maxpool(x) 
        
        x = self.layer1(x) 
        x = self.layer2(x) 
        x = self.layer3(x) 
        x = self.layer4(x) 
        
        x = self.avgpool(x) 
        x = torch.flatten(x,  1)
        x = self.fc(x) 
        return x 
 
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

# Denoising Diffusion Probabilistic Models(DDPM)
The model described in the seminal 2020 paper [Denoising Diffusion Probabilistic Models](https://arxiv.org/abs/2006.11239) (DDPM). For more context, while diffusion models were technically invented [back in 2015](https://arxiv.org/abs/1503.03585)

DDPM is trained quite simply in a few steps:
1. randomly select some timesteps in an iterative noising process.
2. Add noise corresponding to this timestep to the original image. For increasing timesteps, the variance of the noise increases.
3. Pass in this noisy image and the timestep to our model
4. Model is trained with an MSE loss between the model output and the amount of noise added to the image

# DDPM 核心原理算法详解 
 
**Denoising Diffusion Probabilistic Models (DDPM)** 是一种基于马尔可夫链的生成模型，通过逐步加噪和去噪过程实现数据生成。以下是其核心原理算法：
 
## 一、前向扩散过程（加噪）
将原始数据 $x_0$ 逐步添加高斯噪声，经过 $T$ 步转化为纯噪声 $x_T$：
 
$$
q(x_t|x_{t-1}) = \mathcal{N}(x_t; \sqrt{1-\beta_t}x_{t-1}, \beta_t I)
表示在给定前一步状态x_{t-1}的条件下，当前状态x_{t}的分布是一个高斯分布（正态分布）。具体参数如下：\\
均值（mean）： \sqrt{1-\beta_t}x_{t-1}  
方差（variance）：\beta_t I，I是单位矩阵，表示各个维度独立且方差相同
$$
### 1. 均值分量：$\sqrt{1-\beta_t}x_{t-1}$
| **特性**        | **数学意义**               | **物理意义**                     |
|-----------------|--------------------------|----------------------------------|
| 缩减因子        | $\sqrt{1-\beta_t} < 1$   | 对前步数据的缩放衰减             |
| 信号保留        | 与$\beta_t$成反比         | $\beta_t$越大，信号保留越少      |
| 稳定性保障      | 防止数据幅度无限增长       | 维持扩散过程的数值稳定           |
 
### 2. 方差分量：$\beta_t I$
| **特性**        | **数学意义**               | **物理意义**                     |
|-----------------|--------------------------|----------------------------------|
| 各向同性        | $I$ 为单位矩阵           | 各维度独立同分布噪声             |
| 噪声强度        | $\beta_t$ 控制方差大小    | 决定当前步添加的噪声量           |
| 时间依赖性      | $\beta_t$ 随 $t$ 单调递增 | 随时间步增加噪声逐步增强         |
 
## ⚙️ 参数解释
### $\beta_t$ - 噪声调度参数 
- **取值范围**：$(0,1)$  
- **调度特性**：  
  $$\beta_1 < \beta_2 < \cdots < \beta_T$$  
  （典型值：$\beta_1≈0.0001$, $\beta_T≈0.02$）
- **作用机制**：  
  控制噪声注入速率，决定扩散轨迹形状 
 
### $1-\beta_t$ - 信号保留因子
- **物理作用**：  
  平衡信号衰减与噪声注入的比例关系  
  $$\text{信号保留量} \propto \sqrt{1-\beta_t}$$  
  $$\text{噪声注入量} \propto \sqrt{\beta_t}$$


**关键特性：**
1. **闭式表达**：任意时刻 $t$ 的状态可直接从 $x_0$ 计算：
   $$
   x_t = \sqrt{\bar{\alpha}_t}x_0 + \sqrt{1-\bar{\alpha}_t}\epsilon, \quad \epsilon \sim \mathcal{N}(0,I)
   $$
   其中 $\alpha_t = 1-\beta_t$, $\bar{\alpha}_t = \prod_{i=1}^t \alpha_i$
 
2. **噪声调度**：$\beta_t$ 满足 $0 < \beta_1 < \beta_2 < \cdots < \beta_T < 1$，通常采用线性或余弦调度 
 
## 二、反向去噪过程（生成）
学习从噪声 $x_T$ 逐步重建数据 $x_0$ 的逆过程：
 
$$
p_\theta(x_{t-1}|x_t) = \mathcal{N}(x_{t-1}; \mu_\theta(x_t,t), \Sigma_\theta(x_t,t))
$$
 
**参数化形式：**
$$
\mu_\theta(x_t,t) = \frac{1}{\sqrt{\alpha_t}}\left( x_t - \frac{\beta_t}{\sqrt{1-\bar{\alpha}_t}}\epsilon_\theta(x_t,t) \right)
$$
$$
\Sigma_\theta(x_t,t) = \sigma_t^2 I, \quad \sigma_t^2 = \frac{1-\bar{\alpha}_{t-1}}{1-\bar{\alpha}_t}\beta_t
$$
 
## 三、训练目标（简化损失函数）
优化噪声预测网络 $\epsilon_\theta$：
 
$$
\mathcal{L}_{simple} = \mathbb{E}_{t,x_0,\epsilon} \left[ \| \epsilon - \epsilon_\theta(\sqrt{\bar{\alpha}_t}x_0 + \sqrt{1-\bar{\alpha}_t}\epsilon, t) \|^2 \right]
$$
 
**训练流程：**
1. 采样数据 $x_0 \sim q(x_0)$ 
2. 随机选择时间步 $t \sim \text{Uniform}(1,T)$ 
3. 采样噪声 $\epsilon \sim \mathcal{N}(0,I)$
4. 计算梯度下降更新 $\nabla_\theta \|\epsilon - \epsilon_\theta(x_t,t)\|^2$

## 📌 核心采样公式
$$
x_{t-1} = \underbrace{\mu_\theta(x_t, t)}_{\text{确定性成分}} + \underbrace{\sigma_t \cdot z}_{\text{随机成分}},\quad z \sim \mathcal{N}(0,I)
$$
 
## 🔍 双组分物理意义解析
### 🎯 确定性成分 $\mu_\theta(x_t, t)$
| **特性**       | **数学本质**                  | **生成作用**                     |
|----------------|-----------------------------|--------------------------------|
| 去噪方向预测   | 神经网络 $\epsilon_\theta$ 的输出映射 | 指向数据流形的最优重建路径        |
| 信息压缩       | $\mu_\theta = \frac{1}{\sqrt{\alpha_t}}(x_t - \frac{\beta_t}{\sqrt{1-\bar{\alpha}_t}}\epsilon_\theta)$ | 保留有效信号，滤除高频噪声        |
| 分布锚点       | 近似真实后验分布 $q(x_{t-1}\|x_t,x_0)$ 的均值 | 确保生成样本位于数据分布高概率区域 | 

## 四、采样算法（生成新样本）
```python
def ddpm_sampling(model, T):
    x_T = torch.randn_like(data)   # 纯噪声 
    for t in range(T, 0, -1):     # 从T到1反向迭代
        z = torch.randn_like(x_T)  if t > 1 else 0
        # 计算噪声预测
        eps_theta = model(x_T, t) 
        # 计算均值 
        mu_theta = (x_T - (beta_t/sqrt(1-alpha_bar_t))*eps_theta) / sqrt(alpha_t)
        # 更新样本 
        x_{t-1} = mu_theta + sigma_t * z
    return x_0  # 生成样本 

In [ ]:
import torch 
import torch.nn  as nn 
import torch.optim  as optim
import numpy as np 
from torch.utils.data  import DataLoader 
from torchvision import datasets, transforms 
from torchvision.utils  import save_image
import matplotlib.pyplot  as plt
from tqdm import tqdm
 
# ======================
# 配置参数 
# ======================
device = torch.device("cuda"  if torch.cuda.is_available()  else "cpu")
batch_size = 64
image_size = 32  # CIFAR-10标准尺寸
num_epochs = 100
T = 1000  # 扩散步数
beta_start = 1e-4
beta_end = 0.02
 
# ======================
# 噪声调度器 
# ======================
# cosine schedule
def abar(t): return (t*math.pi/2).cos()**2
def inv_abar(x): return x.sqrt().acos()*2/math.pi
def noisify(x0):
    device = x0.device
    n = len(x0)
    t = torch.rand(n,).to(x0).clamp(0,0.999)
    ε = torch.randn(x0.shape, device=device)
    abar_t = abar(t).reshape(-1, 1, 1, 1).to(device)
    xt = abar_t.sqrt()*x0 + (1-abar_t).sqrt()*ε
    return (xt, t.to(device)), ε

class NoiseScheduler:
    def __init__(self, T, beta_start, beta_end):
        self.T = T 
        self.betas  = torch.linspace(beta_start,  beta_end, T).to(device)
        self.alphas  = 1. - self.betas  
        self.alpha_bars  = torch.cumprod(self.alphas,  dim=0)
    
    def add_noise(self, x_0, t, noise=None):
        if noise is None:
            noise = torch.randn_like(x_0) 
        
        alpha_bar_t = self.alpha_bars[t].view(-1,  1, 1, 1)
        noisy_x = torch.sqrt(alpha_bar_t)  * x_0 + torch.sqrt(1  - alpha_bar_t) * noise 
        return noisy_x, noise
 
# ======================
# U-Net模型架构 
# ======================
class Block(nn.Module):
    def __init__(self, in_ch, out_ch, time_emb_dim, up=False):
        super().__init__()
        self.time_mlp  = nn.Linear(time_emb_dim, out_ch)
        if up:
            self.conv  = nn.ConvTranspose2d(in_ch, out_ch, 3, padding=1)
        else:
            self.conv  = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.norm  = nn.BatchNorm2d(out_ch)
        self.act  = nn.SiLU()
    
    def forward(self, x, t):
        h = self.conv(x) 
        time_emb = self.act(self.time_mlp(t)) 
        h = h + time_emb[:, :, None, None]
        h = self.norm(h) 
        return self.act(h) 
 
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=3):
        super().__init__()
        channels = [64, 128, 256, 512]
        
        # 时间嵌入 
        self.time_mlp  = nn.Sequential(
            nn.Linear(1, 32),
            nn.SiLU(),
            nn.Linear(32, 32)
        )
        
        # 下采样路径 
        self.down1  = Block(in_channels, channels[0], 32)
        self.down2  = Block(channels[0], channels[1], 32)
        self.down3  = Block(channels[1], channels[2], 32)
        
        # 最底层
        self.bottleneck  = Block(channels[2], channels[3], 32)
        
        # 上采样路径 
        self.up1  = Block(channels[3] + channels[2], channels[2], 32, up=True)
        self.up2  = Block(channels[2] + channels[1], channels[1], 32, up=True)
        self.up3  = Block(channels[1] + channels[0], channels[0], 32, up=True)
        
        # 输出层 
        self.out  = nn.Conv2d(channels[0], out_channels, 1)
    
    def forward(self, x, t):
        # 时间嵌入
        t = t.unsqueeze(-1).float() 
        t_emb = self.time_mlp(t) 
        
        # 下采样
        d1 = self.down1(x,  t_emb)
        d2 = self.down2(d1,  t_emb)
        d3 = self.down3(d2,  t_emb)
        
        # 瓶颈层 
        bottleneck = self.bottleneck(d3,  t_emb)
        
        # 上采样
        u1 = self.up1(torch.cat([bottleneck,  d3], dim=1), t_emb)
        u2 = self.up2(torch.cat([u1,  d2], dim=1), t_emb)
        u3 = self.up3(torch.cat([u2,  d1], dim=1), t_emb)
        
        return self.out(u3) 
 
# ======================
# 训练函数 
# ======================
def train(model, dataloader, optimizer, scheduler, epochs=T):
    model.train() 
    loss_fn = nn.MSELoss()
    
    for epoch in range(epochs):
        total_loss = 0 
        for images, _ in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            images = images.to(device) 
            
            # 随机采样时间步
            t = torch.randint(0,  T, (images.shape[0],)).to(device) 
            
            # 添加噪声 
            noisy_images, noise = scheduler.add_noise(images,  t)
            
            # 预测噪声 
            predicted_noise = model(noisy_images, t)
            
            # 计算损失 
            loss = loss_fn(predicted_noise, noise)
            
            # 反向传播
            optimizer.zero_grad() 
            loss.backward() 
            optimizer.step() 
            
            total_loss += loss.item() 
        
        avg_loss = total_loss / len(dataloader)
        print(f"Epoch [{epoch+1}/{epochs}] Loss: {avg_loss:.4f}")
        
        # 每10个epoch保存一次模型
        if (epoch + 1) % 10 == 0:
            torch.save(model.state_dict(),  f"ddpm_epoch_{epoch+1}.pth")
    
    return model
 
# ======================
# 采样生成图像 
# ======================
def sample(model, scheduler, n_images=16):
    model.eval() 
    with torch.no_grad(): 
        # 从随机噪声开始
        x = torch.randn(n_images,  3, image_size, image_size).to(device)
        
        for t_step in tqdm(reversed(range(T)), desc="Generating"):
            t = torch.full((n_images,),  t_step, device=device, dtype=torch.long) 
            
            # 预测噪声
            pred_noise = model(x, t)
            
            # 计算系数 
            alpha = scheduler.alphas[t][:,  None, None, None]
            alpha_bar = scheduler.alpha_bars[t][:,  None, None, None]
            beta = scheduler.betas[t][:,  None, None, None]
            
            # 计算重新参数化的均值
            if t_step > 0:
                noise = torch.randn_like(x) 
            else:
                noise = torch.zeros_like(x) 
                
            # DDPM采样公式
            x = (1 / torch.sqrt(alpha))  * (
                x - ((1 - alpha) / (torch.sqrt(1  - alpha_bar))) * pred_noise
            ) + torch.sqrt(beta)  * noise 
            
        # 将图像缩放到[0,1]范围
        x = (x.clamp(-1,  1) + 1) / 2 
        save_image(x, "generated_samples.png",  nrow=4)
        return x

class MixedPrecision():
    def before_fit(self, learn): self.scaler = torch.cuda.amp.GradScaler('cuda')
    def before_batch(self, learn):
        self.autocast = torch.autocast("cuda", dtype=torch.float16)
        self.autocast.__enter__()
    def after_loss(self, learn): self.autocast.__exit__(None, None, None)
    def backward(self, learn): self.scaler.scale(learn.loss).backward()
    def step(self, learn):
        self.scaler.step(learn.opt)
        self.scaler.update()

# ======================
# 主程序 
# ======================
if __name__ == "__main__":
    # 1. 准备数据 
    transform = transforms.Compose([
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    
    # 2. 初始化组件 
    noise_scheduler = NoiseScheduler(T, beta_start, beta_end)
    model = UNet().to(device)
    optimizer = optim.Adam(model.parameters(),  lr=1e-4)
    
    # 3. 训练模型 
    print("Starting training...")
    trained_model = train(model, dataloader, optimizer, noise_scheduler, num_epochs)
    
    # 4. 保存最终模型
    torch.save(trained_model.state_dict(),  "ddpm_final.pth") 
    print("Training completed. Model saved.")
    
    # 5. 生成样本 
    print("Generating samples...")
    generated_images = sample(trained_model, noise_scheduler)

# Denoising Diffusion Implicit Models - DDIM

# DDIM（Denoising Diffusion Implicit Models）核心原理与算法 
 
## 1. 核心思想 
DDIM 是扩散模型的改进形式，通过**非马尔可夫链的确定性过程**实现高质量生成，同时支持**大幅减少采样步骤**（如10-50步）。其关键突破在于：
- 将扩散过程重新参数化为可逆的非马尔可夫链 
- 通过隐式概率模型实现确定性采样 
- 保持与DDPM相同的训练目标（可复用已有模型）
 
## 2. 核心算法步骤 
 
### 2.1 前向过程（非马尔可夫）
定义任意时间序列子集 $\{τ_1,τ_2,...,τ_T\}$，前向过程为：
$$
q_σ(x_{1:T}|x_0) = q_σ(x_T|x_0)\prod_{t=2}^T q_σ(x_{τ_{t-1}}|x_{τ_t},x_0)
$$
其中 $σ$ 控制随机性（当 $σ=0$ 时为确定性过程）
 
### 2.2 反向过程（关键公式）
采样公式：
$$
x_{t-1} = \sqrt{\barα_{t-1}} \left( \frac{x_t - \sqrt{1-\barα_t}ε_θ(x_t,t)}{\sqrt{\barα_t}} \right) + \sqrt{1-\barα_{t-1}-σ_t^2}·ε_θ(x_t,t) + σ_tε 
$$
其中：
- $ε_θ(x_t,t)$：训练好的噪声预测网络 
- $\barα_t$：累积噪声调度参数 
- $σ_t$：控制随机性的超参数 
 
## 3. 关键数学推导 
### 3.1 变分下界重构 
保持与DDPM相同的训练目标：
$$
L_{θ} = \mathbb{E}_{t,x_0,ε}[||ε - ε_θ(x_t,t)||^2]
$$
 
### 3.2 确定性采样（σ=0）
当 $σ_t=0$ 时，采样过程变为：
$$
x_{t-1} = \sqrt{\barα_{t-1}}f_θ(x_t,t) + \sqrt{1-\barα_{t-1}}ε_θ(x_t,t)
$$
其中 $f_θ(x_t,t) = (x_t - \sqrt{1-\barα_t}ε_θ(x_t,t))/\sqrt{\barα_t}$

### DDIM 时间子序列重参数化
DDIM 通过重构原始扩散过程的时间步序列 $\{1,2,...,T\}$ 为任意子序列 $\{\tau_1,\tau_2,...,\tau_S\}$（满足 $S \ll T$），实现跨步计算。核心公式如下：
 
####  🧮 跨步计算公式 
$$x_{\tau_{i-1}} = \sqrt{\bar{\alpha}_{\tau_{i-1}}} \left( \frac{x_{\tau_i} - \sqrt{1-\bar{\alpha}_{\tau_i}} \epsilon_\theta (x_{\tau_i}, \tau_i)} {\sqrt{\bar{\alpha}_{\tau_i}}} \right) + \sqrt{1-\bar{\alpha}_{\tau_{i-1}}} \epsilon_\theta (x_{\tau_i}, \tau_i)$$
 
####  累积噪声调度
$$\bar{\alpha}_{\tau_{i-1}} = \prod_{s=1}^{\tau_{i-1}} (1-\beta_s) \quad \text{(直接跳跃到目标时间步)}$$
 
---
 
###  关键突破点
1. **时间步解耦**  
   - 子序列步 $\tau_i$ 与前一步 $\tau_{i-1}$ 的关系**不依赖中间状态**  
   - 仅通过 $\bar{\alpha}$ 参数关联，实现计算路径的拓扑解耦
 
2. **累积噪声调度**  
   - $\bar{\alpha}$ 的**指数衰减性质**是跨步计算的核心  
   - 通过乘积形式 $\prod_{s=1}^{\tau_{i-1}} (1-\beta_s)$ 直接建模累计噪声影响  
   - 允许在任意子序列步长间进行确定性跳跃

## 4. 与DDPM的关键区别 
| 特性        | DDPM           | DDIM          |
|-------------|----------------|---------------|
| 过程类型    | 马尔可夫链     | 非马尔可夫链  |
| 采样方式    | 随机过程       | 确定性过程    |
| 采样步数    | 通常需要1000+  | 可低至10-50步 |
| 生成轨迹    | 唯一确定       | 多路径可能    |
| 训练兼容性  | 原生模型       | 完全兼容      |
 
## 5. 算法优势 
1. **加速采样**：相比DDPM提速20-100倍 
2. **质量保持**：在10步采样时仍可保持生成质量 
3. **序列插值**：支持潜在空间插值操作 
4. **兼容复用**：可直接使用预训练DDPM模型 
5. **可控随机性**：通过σ参数调节生成多样性 

In [ ]:
import torch 
from math import sqrt 
 
class DDIMSampler(nn.Module):
    def __init__(self, model, timesteps=1000, ddim_steps=50, eta=0.0):
        self.model  = model  # 预训练好的UNet模型 
        self.timesteps  = timesteps 
        self.ddim_steps  = ddim_steps  # 子序列步数(S)
        self.eta  = eta  # 随机因子(η=0时确定性生成)
        
        # 生成时间子序列(τ_1,τ_2,...,τ_S)
        self.time_seq  = self._generate_time_sequence()
        # 预计算累积噪声参数α_bar
        self.alphas_cumprod = model.alphas_cumprod
 
    def _generate_time_sequence(self):
        """生成时间步子序列(等差数列采样)"""
        step_ratio = self.timesteps  // self.ddim_steps  
        return list(range(0, self.timesteps,  step_ratio))[::-1]
 
    def sample_step(self, x, t, t_prev):
        """单步去噪过程 [12]()"""
        # 获取噪声预测模型输出 
        eps_theta = self.model(x,  t)
        
        # 提取α_bar参数 
        alpha_bar = self.alphas_cumprod[t] 
        alpha_bar_prev = self.alphas_cumprod[t_prev] 
        
        # 计算预测的x0 (式(12) [12]())
        pred_x0 = (x - sqrt(1 - alpha_bar) * eps_theta) / sqrt(alpha_bar)
        
        # 计算方差项 
        sigma = self.eta  * sqrt( (1 - alpha_bar_prev)/(1 - alpha_bar) * (1 - alpha_bar/alpha_bar_prev) )
        
        # 计算均值项 
        mean = sqrt(alpha_bar_prev) * pred_x0 + sqrt(1 - alpha_bar_prev - sigma**2) * eps_theta 
        
        # 生成前一步样本 
        if self.eta  == 0:
            return mean 
        else:
            return mean + sigma * torch.randn_like(x) 
 
    def ddim_sample(self, shape=(1,3,64,64)):
        """完整采样流程"""
        x = torch.randn(shape).to(device) 
        
        for i in range(len(self.time_seq)-1): 
            t = self.time_seq[i] 
            t_prev = self.time_seq[i+1] 
            x = self.sample_step(x,  t, t_prev)
        
        return x 

# 牛顿-舒尔茨迭代法（Newton-Schulz Iteration）详解 
 
## 1. 基本原理 
牛顿-舒尔茨迭代法是经典牛顿迭代法在矩阵运算中的扩展，用于求解矩阵平方根问题。其核心思想是通过不动点迭代逼近矩阵平方根，避免了显式的矩阵分解和求逆操作
 
### 数学表达式 
给定非奇异矩阵$A \in \mathbb{R}^{n \times n}$，寻找矩阵$X$使得：
$$ X^2 = A $$
牛顿-舒尔茨迭代公式为：
$$ X_{k+1} = \frac{1}{2} X_k (3I - X_k^2 A) $$
其中$X_0$为初始猜测矩阵，通常取$X_0 = A$或$X_0 = \frac{A}{\|A\|_2}$。

# 生成模型评估指标详解：FID、MMD与KID 
## 1. FID（Fréchet Inception Distance）
### 原理 
- **数学基础**：计算两个多元高斯分布之间的Fréchet距离：
  $$ \text{FID} = \|\mu_1 - \mu_2\|^2 + \text{Tr}(\Sigma_1 + \Sigma_2 - 2(\Sigma_1 \Sigma_2)^{1/2}) $$
  其中 $\mu$ 为均值向量，$\Sigma$ 为协方差矩阵[1]()。
- **特征提取**：使用Inception-v3模型（移除分类层）提取2048维图像特征向量。
- **核心思想**：通过特征空间的统计量（均值、协方差）量化分布差异，值越小表示生成图像与真实图像越相似[2]()。
### 计算步骤 
1. **特征统计**：计算真实图像和生成图像特征的均值向量与协方差矩阵。
2. **矩阵运算**：
   - 计算协方差矩阵乘积的平方根（`linalg.sqrtm` ）
   - 组合均值差平方、协方差迹和平方根迹 
3. **输出**：标量值（`.item()`提取），值越低表示生成质量越好。
### 特点与应用 
- **优势**：与人类视觉感知高度一致，GAN评估的黄金标准[[1][2]()。
- **局限**：假设特征服从高斯分布，对非高斯分布可能高估差异。
- **典型场景**：StyleGAN、ProGAN等生成模型的全局分布评估。
---
## 2. MMD（Maximum Mean Discrepancy）
### 原理 
- **数学基础**：基于再生核希尔伯特空间（RKHS）的均值差异：
  $$ \text{MMD}^2 = \mathbb{E}[k(x,x')] + \mathbb{E}[k(y,y')] - 2\mathbb{E}[k(x,y)] $$
- **核函数**：代码采用三次多项式核 $k(a,b) = (a \cdot b^T / d + 1)^3$，其中 $d$ 为特征维度[7]()。
- **无偏估计**：剔除核矩阵对角线元素（`kxx_sum = kxx.sum()  - kxx.diagonal().sum()` ），避免样本自相似干扰。
### 计算步骤 
1. **核矩阵计算**：同类样本核（`kxx, kyy`）与跨类核（`kxy`）。
2. **统计量组合**：
   - $kxx$ 和 $kyy$ 的剔除对角线和 
   - $kxy$ 的完整和 
3. **标准化输出**：按样本量加权组合结果。
### 特点与应用 
- **优势**：非参数方法，不依赖数据分布假设[7]()。
- **灵活性**：可通过更换核函数（如高斯核）适应不同任务。
- **典型场景**：迁移学习的域适应、双样本假设检验。
---
## 3. KID（Kernel Inception Distance）
### 原理 
- **改进MMD**：通过分块计算MMD的均值，解决大样本计算瓶颈：
  $$ \text{KID} = \frac{1}{n}\sum_{i=1}^n \text{MMD}^2(S_{x,i}, S_{y,i}) $$
- **分块策略**：默认块大小≤50，最小分块数≥4（`n = max(ceil(min(xs/maxs, ys/maxs)), 4)`）[6]()。
- **无偏性**：三次核的无偏估计更符合人类感知[[3][7]()。
### 计算步骤 
1. **动态分块**：根据样本量计算子集数量 $n$。
2. **子集MMD计算**：循环计算每个子集的 `_squared_mmd`。
3. **结果平均**：输出分块MMD的均值（`mmd/n`）。
### 特点与应用 
- **优势**：对小样本更鲁棒，计算效率高于FID[6]()。
- **典型场景**：评估生成图像的局部特征一致性（如StyleGAN细节纹理）。
---
## 指标对比总结 
| **指标** | **理论基础**       | **计算复杂度** | **数据假设** | **核心优势**               |
|----------|---------------------|----------------|--------------|----------------------------|
| FID      | Fréchet距离         | 高（矩阵分解） | 高斯分布     | 全局分布评估               |
| MMD      | 核空间均值差异      | 中             | 无           | 非参数、灵活核函数         |
| KID      | MMD分块平均         | 低             | 无           | 局部特征一致性、小样本鲁棒 |
> **应用建议**：  
> - 需快速评估全局质量 → **FID**  
> - 关注局部细节或无分布假设 → **KID**  
> - 非图像数据或迁移学习 → **MMD**

In [ ]:
def _sqrtm_newton_schulz(mat, num_iters=100):
    mat_nrm = mat.norm()
    mat = mat.double()
    Y = mat/mat_nrm
    n = len(mat)
    I = torch.eye(n, n).to(mat)
    Z = torch.eye(n, n).to(mat)

    for i in range(num_iters):
        T = (3*I - Z@Y)/2
        Y,Z = Y@T,T@Z
        res = Y*mat_nrm.sqrt()
        if ((mat-(res@res)).norm()/mat_nrm).abs()<=1e-6: break
    return res

def _calc_stats(feats):
    feats = feats.squeeze()
    return feats.mean(0),feats.T.cov()

def _calc_fid(m1,c1,m2,c2):
#     csr = _sqrtm_newton_schulz(c1@c2)
    csr = tensor(linalg.sqrtm(c1@c2, 256).real)
    return (((m1-m2)**2).sum() + c1.trace() + c2.trace() - 2*csr.trace()).item()

def _squared_mmd(x, y):
    def k(a,b): return (a@b.transpose(-2,-1)/a.shape[-1]+1)**3
    m,n = x.shape[-2],y.shape[-2]
    kxx,kyy,kxy = k(x,x), k(y,y), k(x,y)
    kxx_sum = kxx.sum([-1,-2])-kxx.diagonal(0,-1,-2).sum(-1)
    kyy_sum = kyy.sum([-1,-2])-kyy.diagonal(0,-1,-2).sum(-1)
    kxy_sum = kxy.sum([-1,-2])
    return kxx_sum/m/(m-1) + kyy_sum/n/(n-1) - kxy_sum*2/m/n

def _calc_kid(x, y, maxs=50):
    xs,ys = x.shape[0],y.shape[0]
    n = max(math.ceil(min(xs/maxs, ys/maxs)), 4)
    mmd = 0.
    for i in range(n):
        cur_x = x[round(i*xs/n) : round((i+1)*xs/n)]
        cur_y = y[round(i*ys/n) : round((i+1)*ys/n)]
        mmd += _squared_mmd(cur_x, cur_y)
    return (mmd/n).item()

class ImageEval:
    def __init__(self, model, dls, cbs=None):
        self.learn = TrainLearner(model, dls, loss_func=fc.noop, cbs=cbs, opt_func=None)
        self.feats = self.learn.capture_preds()[0].float().cpu().squeeze()
        self.stats = _calc_stats(self.feats)

    def get_feats(self, samp):
        self.learn.dls = DataLoaders([],[(samp, tensor([0]))])
        return self.learn.capture_preds()[0].float().cpu().squeeze()

    def fid(self, samp): return _calc_fid(*self.stats, *_calc_stats(self.get_feats(samp)))
    def kid(self, samp): return _calc_kid(self.feats, self.get_feats(samp))


# Karras pre-conditioning
Karras Scaling算法的核心思想
Karras Scaling算法是一种用于稳定和加速生成模型（特别是扩散模型）训练的技术，其核心思想是通过动态调整网络层的输入输出尺度来优化训练过程。该算法基于对神经网络训练动态的深入分析，发现不同层级的激活值尺度会显著影响模型的收敛性和最终性能。

算法数学表达
Karras Scaling算法通过以下公式实现网络层的尺度控制： 对于给定的神经网络层，设权重矩阵为W，偏置向量为b，则经过Scaling变换后的参数为： W' = λW b' = λb 其中λ是一个与训练时间步相关的比例因子，用于控制Scaling的程度。这个比例因子通常设计为随时间步变化的函数，使得网络在不同训练阶段具有不同的动态特性。

# Karras Scaling算法在扩散模型中的应用解析 
 
## 时间条件化实现机制 
**核心公式**：  
`fθ(xt,t) = c_skip(t)x_t + c_out(t)Fθ(c_in(t)x_t, c_noise(t))`  

**损失函数loss**:

![Various norms](../imgs/karras.png)

**关键系数函数**：
| 函数       | 作用描述                           | 设计目标                     |
|------------|----------------------------------|-----------------------------|
| `c_skip(t)` | 保留原始输入的跳跃连接系数          | 维持信号完整性                |
| `c_out(t)`  | 控制网络输出的缩放因子              | 调节预测信号的强度            |
| `c_in(t)`   | 输入特征归一化系数                 | 稳定不同噪声水平的输入分布     |
| `c_noise(t)`| 时间嵌入向量的调节系数             | 动态平衡时间条件的影响         |
 
## 扩散模型中的具体应用 
1. **输入归一化**  
   - 通过`c_in(t)`动态调整噪声输入尺度，消除不同采样步间的量级差异
 
2. **输出控制**  
   - `c_out(t)`将网络输出映射到合适的信号范围，避免幅值震荡 
 
3. **残差连接**  
   - `c_skip(t)`构建的跳跃连接保留低频信息，防止高频细节丢失
 
4. **时间嵌入优化**  
   - `c_noise(t)`加权时间编码，增强模型对噪声调制的敏感性 

# Karras Scaling系数系统详解（2025年最新整理）
噪声样本的方差 $Var(x_t)= \sigma_t^2 Var(x_{0}) + \alpha_t^2 Var(\varepsilon) = \sigma_t^2 + \alpha_t^2$ 假设数据已经标准化

## 核心系数定义表 
| 系数          | 数学形式                          | 物理意义                                                                 | 动态范围分析                |
|---------------|----------------------------------|--------------------------------------------------------------------------|---------------------------|
| **c_in(t)**   | $\frac{1}{\sqrt{\sigma_t^2 + \alpha_t^2}}$ | 输入特征标准化<br>• 消除量纲差异<br>• 稳定反向传播梯度                    | 随$t$增大单调递减           |
| **c_out(t)**  | $\frac{\sigma_t}{\sqrt{\sigma_t^2 + \alpha_t^2}}$ | 输出尺度校准<br>• 将网络输出$F_\theta$映射到噪声空间<br>• 控制高频细节生成 | 从$0$→$1$单调递增           |
| **c_skip(t)** | $\frac{\alpha_t}{\sqrt{\sigma_t^2 + \alpha_t^2}}$ | 残差连接门控<br>• 保留低频信号成分<br>• 防止梯度消失                      | 从$1$→$0$单调递减           |
 
## 关键技术原理 
 
### 1. 协同设计约束 
- **方差守恒定理**：  
  $\text{c\_skip}^2(t) + \text{c\_out}^2(t) \equiv 1$  
  → 确保网络输出$\epsilon_\theta$的方差与真实噪声$\epsilon$匹配 
 
- **渐进特性**：  
  ```python 
  # 伪代码演示系数动态变化
  def coefficient_behavior(t, T):
      c_skip = alpha_t / sqrt(sigma_t**2 + alpha_t**2)  # 随t增加→0
      c_out = sigma_t / sqrt(sigma_t**2 + alpha_t**2)   # 随t增加→1
      return c_skip, c_out 

## 算法优势分析 
**训练稳定性**  
- 通过系数函数的边界控制，有效抑制梯度异常（如爆炸/消失问题）
 
**收敛加速**  
- 实验表明可减少30%-50%训练迭代次数（基于EDM论文数据）
 
**生成质量提升**  
- 在FFHQ数据集测试中，FID分数平均改善15%-20%
 
**架构兼容性**  
- 已验证适配UNet、Transformer等多种骨干网络


为什么需要多次采样？——扩散模型的迭代去噪本质
扩散模型生成图像的核心是逐步去噪的迭代过程：
- 单次去噪的局限性：denoise()函数确实能预测噪声，但直接用它生成的"去噪结果"只是当前步骤的局部最优解，可能包含残留噪声或结构错误。
- 渐进修正机制：通过多次采样（如20~30步），模型可以：
- 逐步修正错误：前一步的误差会在下一步被重新评估和调整（类似画家反复修改草稿）。
- 控制收敛稳定性：大噪声步（早期）粗调整体结构，小噪声步（后期）细化细节


# 扩散模型采样方法详解
 
## 欧拉采样方法（Euler）
欧拉采样器是最简单的数值解法，基于经典ODE求解器原理。其核心思想是用差商近似微分，通过当前步的梯度直接线性外推下一步的值。算法公式可表示为：`x_{t-1} = x_t + Δt * f(x_t, t)`，其中Δt是步长，f(x_t, t)是噪声预测函数。欧拉方法计算量小但精度一般，对初始条件敏感，容易出现数值不稳定现象。在扩散模型中，它通过噪声计划逐步减少噪声量，最终生成清晰图像。
 
## 欧拉祖先采样器（Euler a）
欧拉祖先采样器是带有随机性的改进版本，属于祖先采样器类别。其核心原理是在每个采样步骤中额外添加随机噪声，公式为：`x_{t-1} = x_t + Δt * f(x_t, t) + σ_t * ε`，其中ε是从标准正态分布采样的噪声。这种设计使得生成结果具有不可收敛的特性，每次采样都会产生随机偏差，适合需要创造性和多样性的场景。与标准欧拉法相比，它在去噪过程中会减去"比应该更多"的噪声再补入新噪声。
 
## Heun方法 
Heun方法是欧拉法的二阶改进版本，采用预测-校正策略提高精度。其算法分为两步：
1. 首先计算预测值：`y* = x_t + Δt * f(x_t, t)`
2. 然后校正步骤：`x_{t-1} = x_t + Δt/2 * [f(x_t, t) + f(y*, t+Δt)]`
 
核心原理是通过两次函数评估来获得更准确的斜率估计，虽然计算量是欧拉法的两倍，但能显著减少截断误差。在扩散模型中，Heun尤其适合需要高质量图像生成的场景。 
 
## 线性多步法（LMS）
线性多步法（LMS）属于经典ODE求解器，其核心原理是利用前几步的历史信息来提高当前步的估计精度。基本公式为：`x_{t-1} = x_t + Δt * Σ(α_i * f(x_{t-i}, t-i))`，其中α_i是加权系数。与欧拉法相比，LMS通过多步信息交换获得更稳健的求解效果，在保持相同计算速度的同时提供更高准确性。在扩散模型中，LMS特别适合需要平衡速度与质量的场景，能有效抑制数值振荡。


In [ ]:
# sig_data = xb.std()
sig_data = 0.66

def scalings(sig):
    totvar = sig**2+sig_data**2
    # c_skip,c_out,c_in
    return sig_data**2/totvar,sig*sig_data/totvar.sqrt(),1/totvar.sqrt()

def noisify(x0):
    device = x0.device
    # sig = (0.3 + 0.7*torch.rand([len(x0)])).log().to(x0)   # 更稳定的分布，避免极端值
    sig = (torch.randn([len(x0)])*1.2-1.2).exp().to(x0).reshape(-1,1,1,1)
    noise = torch.randn_like(x0, device=device)
    c_skip,c_out,c_in = scalings(sig)
    noised_input = x0 + noise*sig
    target = (x0-c_skip*noised_input)/c_out
    return (noised_input*c_in,sig.squeeze()),target

sz = (2048,1,32,32)

def denoise(model, x, sig):
    c_skip,c_out,c_in = scalings(sig)
    return model((x*c_in, sig))*c_out + x*c_skip

def get_ancestral_step(sigma_from, sigma_to, eta=1.):
    if not eta: return sigma_to, 0.
    var_to,var_from = sigma_to**2,sigma_from**2
    sigma_up = min(sigma_to, eta * (var_to * (var_from-var_to)/var_from)**0.5)
    return (var_to-sigma_up**2)**0.5, sigma_up

@torch.no_grad()
def sample_euler_ancestral(x, sigs, i, model, eta=1.):
    sig,sig2 = sigs[i],sigs[i+1]
    denoised = denoise(model, x, sig)
    sigma_down,sigma_up = get_ancestral_step(sig, sig2, eta=eta)
    x = x + (x-denoised)/sig*(sigma_down-sig)
    return x + torch.randn_like(x)*sigma_up

@torch.no_grad()
def sample_euler(x, sigs, i, model):
    sig,sig2 = sigs[i],sigs[i+1]
    denoised = denoise(model, x, sig)
    return x + (x-denoised)/sig*(sig2-sig)

def sigmas_karras(n, sigma_min=0.01, sigma_max=80., rho=7., device='cpu'):
    ramp = torch.linspace(0, 1, n)
    min_inv_rho = sigma_min**(1/rho)
    max_inv_rho = sigma_max**(1/rho)
    sigmas = (max_inv_rho + ramp * (min_inv_rho-max_inv_rho))**rho
    return torch.cat([sigmas, tensor([0.])]).to(device)

@torch.no_grad()
def sample_heun(x, sigs, i, model, s_churn=0., s_tmin=0., s_tmax=float('inf'), s_noise=1.):
    sig,sig2 = sigs[i],sigs[i+1]
    n = len(sigs)
    gamma = min(s_churn/(n-1), 2**0.5-1) if s_tmin<=sig<=s_tmax else 0.
    eps = torch.randn_like(x) * s_noise
    sigma_hat = sig * (gamma+1)
    if gamma > 0: x = x + eps * (sigma_hat**2-sig**2)**0.5
    denoised = denoise(model, x, sig)
    d = (x-denoised)/sig
    dt = sig2-sigma_hat
    x_2 = x + d*dt
    if sig2==0: return x_2
    denoised_2 = denoise(model, x_2, sig2)
    d_2 = (x_2-denoised_2)/sig2
    d_prime = (d+d_2)/2
    return x + d_prime*dt

def sample(sampler, model, steps=100, sigma_max=80., **kwargs):
    preds = []
    x = torch.randn(sz).to(model.device)*sigma_max
    sigs = sigmas_karras(steps, device=model.device, sigma_max=sigma_max)
    for i in range(len(sigs)-1):
        x = sampler(x, sigs, i, model, **kwargs)
        preds.append(x)
    return preds

def linear_multistep_coeff(order, t, i, j):
    if order-1 > i: raise ValueError(f'Order {order} too high for step {i}')
    def fn(tau):
        prod = 1.
        for k in range(order):
            if j == k: continue
            prod *= (tau-t[i-k]) / (t[i-j]-t[i-k])
        return prod
    return integrate.quad(fn, t[i], t[i+1], epsrel=1e-4)[0]

@torch.no_grad()
def sample_lms(model, steps=100, order=4, sigma_max=80.):
    preds = []
    x = torch.randn(sz).to(model.device)*sigma_max
    sigs = sigmas_karras(steps, device=model.device, sigma_max=sigma_max)
    ds = []
    for i in range(len(sigs)-1):
        sig = sigs[i]
        denoised = denoise(model, x, sig)
        d = (x-denoised)/sig
        ds.append(d)
        if len(ds) > order: ds.pop(0)
        cur_order = min(i+1, order)
        coeffs = [linear_multistep_coeff(cur_order, sigs, i, j) for j in range(cur_order)]
        x = x + sum(coeff*d for coeff, d in zip(coeffs, reversed(ds)))
        preds.append(x)
    return preds

# 交叉卷积（Cross Convolution）技术详解 
 
## 核心定义
交叉卷积是一种改进的卷积操作范式，通过特征交叉机制增强传统卷积的特征交互能力。
"CrossConv"的命名主要源自其**跨特征交互**的设计理念：
1. **跨层连接**：通过ResBlock实现特征跨层融合（跳跃连接）
2. **跨感受野**：组合残差块与标准卷积形成混合感受野 
3. **跨模态处理**：在特征空间中实现不同抽象层次的信息交叉 
 
## 典型结构对比 
| 模块类型     | 连接方式              | 特征交互模式       |
|--------------|-----------------------|--------------------|
| 普通卷积      | 纯序列化              | 单向流动           |
| **CrossConv** | 残差+卷积交叉         | 双向特征融合       |
| DenseBlock   | 密集连接              | 全互连             |
 

In [ ]:
def cross_conv(nf, act, norm):
    return nn.Sequential(
        ResBlock(nf, nf, act=act, norm=norm),
        nn.Conv2d(nf, nf, 3, padding=1)
    )

class TinyUnet(nn.Module):
    def __init__(self, act=act_gr, nfs=(32,64,128,256,512,1024), norm=nn.BatchNorm2d):
        super().__init__()
#         初始化模型的各个部分：
# self.start ：一个残差块，用于处理输入图像，输入通道数为 3，输出通道数为 nfs[0]。
# self.dn ：一个 nn.ModuleList，包含多个残差块，用于下采样过程，每个残差块的步长为 2。
# self.xs ：一个 nn.ModuleList，包含多个 cross_conv 模块，用于跨连接。
# self.up ：一个 nn.ModuleList，包含多个上采样块（up_block）和一个残差块，用于上采样过程。
# self.end ：一个残差块，用于输出最终结果，输入和输出通道数均为 3。
        self.start = ResBlock(3, nfs[0], ks=5, stride=1, act=act, norm=norm)
        self.dn = nn.ModuleList([ResBlock(nfs[i], nfs[i+1], act=act, norm=norm, stride=2)
                                 for i in range(len(nfs)-1)])
        self.xs = nn.ModuleList([cross_conv(nfs[i], act, norm)
                                 for i in range(len(nfs)-1,0,-1)])
        self.xs += [cross_conv(nfs[0], act, norm)]
        self.up = nn.ModuleList([up_block(nfs[i], nfs[i-1], act=act, norm=norm)
                                 for i in range(len(nfs)-1,0,-1)])
        self.up += [ResBlock(nfs[0], 3, act=act, norm=norm)]
        self.end = ResBlock(3, 3, act=nn.Identity, norm=norm)

    def forward(self, x):
        # 将输入 x 存储在 layers 列表中。
        # 通过 self.start 处理输入。
        # 进行下采样过程，将每个下采样阶段的输出存储在 layers 列表中。
        # 进行上采样过程，在每个上采样阶段，将跨连接的输出加到当前特征图上。
        # 将最终特征图与输入相加，并通过 self.end 输出最终结果。
        layers = []
        layers.append(x)
        x = self.start(x)
        for i,l in enumerate(self.dn):
            layers.append(x)
            x = l(x)
        n = len(layers)
        for i,l in enumerate(self.up):
            if i!=0: x += self.xs[i](layers[n-i])
            x = l(x)
        return self.end(x+layers[0])

# class TfmDS:
#     def __init__(self, ds, tfmx=fc.noop, tfmy=fc.noop): self.ds,self.tfmx,self.tfmy = ds,tfmx,tfmy
#     def __len__(self): return len(self.ds)
#     def __getitem__(self, i):
#         item = self.ds[i]
#         return self.tfmx(item),self.tfmy(item)

# def denorm(x): return (x*xstd[:,None,None]+xmean[:,None,None]).clamp(0,1)

# def tfmx(x, erase=True):
#     x = TF.resize(x, (32,32))[None]
#     x = F.interpolate(x, scale_factor=2)
#     if erase: x = rand_erase(x)
#     return x[0]

# tds = TinyDS(path/'train')
# vds = TinyDS(path/'val')

# tfm_tds = TfmDS(tds, tfmx)
# tfm_vds = TfmDS(vds, partial(tfmx, erase=False))

# dls = DataLoaders(*get_dls(tfm_tds, tfm_vds, bs=bs, num_workers=0))

In [ ]:
class SelfAttention(nn.Module):
    def __init__(self, ni):
        super().__init__()
        self.scale = math.sqrt(ni)#假设每个元素为iid(mean = 0, var = 1)，则QK^T的方差为ni，所以此处缩放回1
        self.norm = nn.GroupNorm(1, ni)
        self.q = nn.Linear(ni, ni)
        self.k = nn.Linear(ni, ni)
        self.v = nn.Linear(ni, ni)
        self.proj = nn.Linear(ni, ni)
    
    def forward(self, x):
        inp = x
        n,c,h,w = x.shape
        x = self.norm(x)
        x = x.view(n, c, -1).transpose(1, 2)
        q = self.q(x)
        k = self.k(x)
        v = self.v(x)
        s = (q@k.transpose(1,2))/self.scale
        x = s.softmax(dim=-1)@v #此处-1仅表示使用dim = -1进行归一化，即对每行的 Key 分数进行归一化，@v对Value向量进行加权求和
        x = self.proj(x)
        x = x.transpose(1,2).reshape(n,c,h,w)
        return x+inp

def cp_parms(a,b):
    b.weight = a.weight
    b.bias = a.bias

sa = SelfAttention(32)

# 初始化 Attention 模块 
at = Attention(
    query_dim=32,  # 输入维度 
    heads=1,  # 注意力头数 
    dim_head=32,  # 每个头的维度 
    cross_attention_norm = "group_norm",
    norm_num_groups=1  # 归一化组数 
)
 
# 源和目标属性 
src = sa.q, sa.k, sa.v, sa.proj,  sa.norm  
dst = at.to_q,  at.to_k,  at.to_v,  at.to_out,  at.norm_cross  
 
# 复制参数 
for s, d in zip(src, dst):
    print(s,d)
    cp_parms(s, d)


class SelfAttentionMultiHead(nn.Module):
    def __init__(self, ni, nheads):
        super().__init__()
        self.nheads = nheads
        self.scale = math.sqrt(ni/nheads)
        self.norm = nn.BatchNorm2d(ni)
        self.qkv = nn.Linear(ni, ni*3)
        self.proj = nn.Linear(ni, ni)
    
    def forward(self, inp):
        n,c,h,w = inp.shape
        x = self.norm(inp).view(n, c, -1).transpose(1, 2)
        x = self.qkv(x)
        x = rearrange(x, 'n s (h d) -> (n h) s d', h=self.nheads)
        q,k,v = torch.chunk(x, 3, dim=-1)
        s = (q@k.transpose(1,2))/self.scale
        x = s.softmax(dim=-1)@v
        x = rearrange(x, '(n h) s d -> n s (h d)', h=self.nheads)
        x = self.proj(x).transpose(1,2).reshape(n,c,h,w)
        return x+inp
 
class CrossAttention(nn.Module):
    def __init__(self, dim, nheads):
        super().__init__()
        self.nheads  = nheads 
        self.scale  = math.sqrt(dim  / nheads)
        
        # Query 来自输入序列A，Key/Value 来自序列B
        self.to_q  = nn.Linear(dim, dim)  # 仅对Query做投影 
        self.to_kv  = nn.Linear(dim, dim * 2)  # 同时生成Key和Value 
        
        self.proj  = nn.Linear(dim, dim)  # 输出投影层 
 
    def forward(self, x, context):
        """
        Args:
            x:      Query序列 [batch, seq_len_q, dim]
            context: Key/Value序列 [batch, seq_len_kv, dim]
        Returns:
            输出序列 [batch, seq_len_q, dim]
        """
        b, nq, d = x.shape  
        _, nkv, _ = context.shape  
        
        # 1. 生成Q/K/V
        q = self.to_q(x)   # [b, nq, dim]
        kv = self.to_kv(context)   # [b, nkv, dim*2]
        k, v = torch.chunk(kv,  2, dim=-1)  # 各[b, nkv, dim]
        
        # 2. 多头拆分 
        q = rearrange(q, 'b nq (h d) -> (b h) nq d', h=self.nheads)   # [b*h, nq, d_h]
        k = rearrange(k, 'b nkv (h d) -> (b h) nkv d', h=self.nheads)   # [b*h, nkv, d_h]
        v = rearrange(v, 'b nkv (h d) -> (b h) nkv d', h=self.nheads)   # [b*h, nkv, d_h]
        
        # 3. 计算注意力分数 
        attn = (q @ k.transpose(-2,  -1)) / self.scale   # [b*h, nq, nkv]
        attn = attn.softmax(dim=-1) 
        
        # 4. 加权求和
        out = attn @ v  # [b*h, nq, d_h]
        out = rearrange(out, '(b h) nq d -> b nq (h d)', h=self.nheads)   # [b, nq, dim]
        
        # 5. 输出投影 
        return self.proj(out) 